![image](https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/.github/pytorch3dlogo.png)

    PyTorch3D provides efficient, reusable components for 3D Computer Vision research with PyTorch.

    Key features include:
        - Data structure for storing and manipulating triangle meshes, pointclouds, virtual cameras, volumes and so on
        - Efficient operations on triangle meshes (projective transformations, graph convolution, sampling, loss functions)
        - A differentiable mesh and pointcloud renderer

In [ ]:
from pathlib import Path

import torch
import pytorch3d
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
assert pytorch3d.__version__.startswith('0.6')
assert torch.__version__.startswith('1.9.1')

# 3D Structures

In [ ]:
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.structures import Pointclouds, Meshes
from pytorch3d.vis.plotly_vis import plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    FoVPerspectiveCameras,
    PointsRasterizer,
    PointsRasterizationSettings,
    PointsRenderer,
    AlphaCompositor,
    RasterizationSettings,
    PointLights,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader
)

DEVICE = torch.device('cuda:1')

## Pointclouds

In [ ]:
!mkdir -p data/PittsburghBridge
!wget -P data/PittsburghBridge https://dl.fbaipublicfiles.com/pytorch3d/data/PittsburghBridge/pointcloud.npz

In [ ]:
obj_filename = Path('data/PittsburghBridge/pointcloud.npz')

raw_pointcloud = np.load(obj_filename)

vertexes = torch.from_numpy(raw_pointcloud['verts'])
rgb = torch.from_numpy(raw_pointcloud['rgb'])
pointcloud = Pointclouds(points=[vertexes], features=[rgb]).to(DEVICE)

In [ ]:
plot_scene({
    'scene': {
        'pointcloud': pointcloud
    }
})

## Render Pointclouds

In [ ]:
R, T = look_at_view_transform(20, 10, 0)
cameras = FoVOrthographicCameras(device=DEVICE, R=R, T=T, znear=0.01)

### Feel free to rotate the camera position in plotly

In [ ]:
plot_scene({
    'scene': {
        'pointcloud': pointcloud,
        'camera': cameras
    }
})

In [ ]:
# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
raster_settings = PointsRasterizationSettings(
    image_size=512,
    radius=0.003,
    points_per_pixel=10
)


# Create a points renderer by compositing points using an alpha compositor (nearer points
# are weighted more heavily).
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

In [ ]:
images = renderer(pointcloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")

## Meshes

In [ ]:
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png

In [ ]:
obj_filename = Path('data/cow_mesh/cow.obj')
mesh = load_objs_as_meshes([obj_filename], device=DEVICE)

### Only mesh

In [ ]:
plot_scene({
    'scene': {
        'mesh': mesh,
    }
})

### Texture map and UV points

In [ ]:
plt.figure(figsize=(7,7))
texturesuv_image_matplotlib(mesh.textures, subsample=None)
plt.axis("off");

In [ ]:
# Initialize a camera.
# With world coordinates +Y up, +X left and +Z in, the front of the cow is facing the -Z direction.
# So we move the camera by 180 in the azimuth direction so it is facing the front of the cow.
R, T = look_at_view_transform(2.7, 0, 180)
cameras = FoVPerspectiveCameras(device=DEVICE, R=R, T=T)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. We also set bin_size and max_faces_per_bin to None which ensure that
# the faster coarse-to-fine rasterization method is used. Refer to rasterize_meshes.py for
# explanations of these parameters. Refer to docs/notes/renderer.md for an explanation of
# the difference between naive and coarse-to-fine rasterization.
raster_settings = RasterizationSettings(
    image_size=512,
    blur_radius=0.0,
    faces_per_pixel=1,
)

# Place a point light in front of the object. As mentioned above, the front of the cow is facing the
# -z direction.
lights = PointLights(device=DEVICE, location=[[0.0, 0.0, -3.0]])

# Create a Phong renderer by composing a rasterizer and a shader. The textured Phong shader will
# interpolate the texture uv coordinates for each vertex, sample from a texture image and
# apply the Phong lighting model
renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras,
        raster_settings=raster_settings
    ),
    shader=SoftPhongShader(
        device=DEVICE,
        cameras=cameras,
        lights=lights
    )
)

In [ ]:
images = renderer(mesh)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off");

---

# Cameras and Rays

In [ ]:
from pytorch3d.renderer import FoVOrthographicCameras, RayBundle

In [ ]:
origin_camera = FoVPerspectiveCameras()

# Rays that are used, for example, in NeRF
origin_rays = RayBundle(
    origins=torch.zeros(100, 3),
    directions=torch.nn.functional.normalize(torch.rand(100, 3), dim=-1),
    lengths=torch.ones(100, 3),
    xys=torch.tensor([1]).view(1, 1)
)

In [ ]:
plot_scene({
    'scene': {
        'base_camera': origin_camera,
        'ray': origin_rays
    }
})

## Translate and Rotate Camera

In [ ]:
from pytorch3d.transforms import Translate, Rotate, random_rotation, Transform3d
from pytorch3d.renderer.cameras import CamerasBase

In [ ]:
camera_transform = (
    Transform3d()
    .translate(1, 1, 1)
    .rotate(random_rotation())
)

In [ ]:
origin_camera_matrix = Transform3d().rotate(origin_camera.R).translate(origin_camera.T)

In [ ]:
result_camera_matrix = camera_transform.compose(origin_camera_matrix)
result_R = result_camera_matrix.get_matrix()[..., :3, :3]
result_T = result_camera_matrix.get_matrix()[..., 3, :3]

new_camera = FoVPerspectiveCameras(R=result_R, T=result_T, )

In [ ]:
plot_scene({
    'scene': {
        'base_camera': origin_camera,
        'transformed_camera': new_camera
    }
})